# Yelp API Calls

Anjali Prakash

**We will be using Yelp API calls to compile all bagel stores in New York into a single data frame**



In [3]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook


In [5]:
# Load API Credentials
with open('/Users/anjali_work/.secret/yelp_api.json') as f:   #use your path here!
    login = json.load(f)
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

In [7]:
# set our API call parameters 
LOCATION = 'NY,NY'
TERM = 'Bagels'

In [8]:
# Specifying JSON_FILE filename 
JSON_FILE = "Data/results_in_progress_NY_bagels.json"
JSON_FILE

'Data/results_in_progress_NY_bagels.json'

In [9]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_NY_bagels.json not found. Saving empty list to file.


In [10]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


In [11]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [12]:
## How many results total?
total_results = results['total']
total_results

8100

In [13]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

In [14]:
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

405

In [16]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

In [17]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [18]:
from tqdm.notebook import tqdm_notebook
import time
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 

  0%|          | 0/405 [00:00<?, ?it/s]

In [19]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    # add a 200ms pause
    time.sleep(.2)

  0%|          | 0/405 [00:00<?, ?it/s]

YelpAPIError: VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.

In [20]:
## delete file and confirm it no longer exits.
os.remove(JSON_FILE)
os.path.isfile(JSON_FILE)

False

In [21]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)  

In [22]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

[i] Data/results_in_progress_NY_bagels.json not found. Saving empty list to new file.
- 0 previous results found.


405

In [23]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/405 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


In [24]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,j1S3NUrkB3BVT49n_e76NQ,best-bagel-and-coffee-new-york,Best Bagel & Coffee,https://s3-media1.fl.yelpcdn.com/bphoto/91RtGG...,False,https://www.yelp.com/biz/best-bagel-and-coffee...,4563,"[{'alias': 'bagels', 'title': 'Bagels'}, {'ali...",4.5,"{'latitude': 40.7522683, 'longitude': -73.9910...","[pickup, delivery]",$,"{'address1': '225 W 35th St', 'address2': '', ...",+12125644409,(212) 564-4409,5213.531501
1,VrCCr45dhN-RRM107iptdg,russ-and-daughters-new-york,Russ & Daughters,https://s3-media2.fl.yelpcdn.com/bphoto/fZTO8K...,False,https://www.yelp.com/biz/russ-and-daughters-ne...,2793,"[{'alias': 'bagels', 'title': 'Bagels'}, {'ali...",4.5,"{'latitude': 40.722589, 'longitude': -73.98828...",[delivery],$$,"{'address1': '179 E Houston St', 'address2': '...",+12124754880,(212) 475-4880,1972.496648
2,foO2N-TrdPBO-dFn6M35TA,brooklyn-bagel-and-coffee-company-new-york-8,Brooklyn Bagel & Coffee Company,https://s3-media2.fl.yelpcdn.com/bphoto/P5mdzj...,False,https://www.yelp.com/biz/brooklyn-bagel-and-co...,283,"[{'alias': 'bagels', 'title': 'Bagels'}, {'ali...",4.5,"{'latitude': 40.73104, 'longitude': -73.9932199}","[pickup, delivery]",$,"{'address1': '63 E 8th St', 'address2': None, ...",+12124773070,(212) 477-3070,2846.031390
3,YuirIY8ukMb5F8BQhHh82A,pop-up-bagels-new-york,Pop Up Bagels,https://s3-media4.fl.yelpcdn.com/bphoto/Halg-t...,False,https://www.yelp.com/biz/pop-up-bagels-new-yor...,90,"[{'alias': 'bagels', 'title': 'Bagels'}, {'ali...",4.5,"{'latitude': 40.72792, 'longitude': -74.00022}",[],NaN,"{'address1': '177 Thompson St', 'address2': ''...",,,2549.322239
4,oi39VAwo4-KGm_gSkWPCsQ,tompkins-square-bagels-avenue-a-new-york,Tompkins Square Bagels - Avenue A,https://s3-media3.fl.yelpcdn.com/bphoto/VbTibL...,False,https://www.yelp.com/biz/tompkins-square-bagel...,1502,"[{'alias': 'bagels', 'title': 'Bagels'}]",4.0,"{'latitude': 40.72823, 'longitude': -73.9823}","[pickup, delivery]",$$,"{'address1': '165 Avenue A', 'address2': '', '...",+16463516520,(646) 351-6520,2727.847810


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,dKgMlvm_kcm6eLxCjkCwGw,forest-superette-and-bagels-staten-island,Forest superette and bagels,https://s3-media4.fl.yelpcdn.com/bphoto/vQytzM...,False,https://www.yelp.com/biz/forest-superette-and-...,7,"[{'alias': 'bagels', 'title': 'Bagels'}]",3.0,"{'latitude': 40.6308999, 'longitude': -74.10101}","[pickup, delivery]",$,"{'address1': '350 Forest Ave', 'address2': Non...",+17182736928,(718) 273-6928,12234.759914
996,CVxz9B5ncpMwVK1bk_c1YA,veselka-new-york,Veselka,https://s3-media3.fl.yelpcdn.com/bphoto/19rGb5...,False,https://www.yelp.com/biz/veselka-new-york?adju...,3086,"[{'alias': 'diners', 'title': 'Diners'}, {'ali...",4.0,"{'latitude': 40.729036, 'longitude': -73.98712}","[pickup, delivery]",$$,"{'address1': '144 2nd Ave', 'address2': '', 'a...",+12122289682,(212) 228-9682,2691.973201
997,kCRNgd0gE1jqdxH6iKRjRw,hidden-grounds-coffee-jersey-city-4,Hidden Grounds Coffee,https://s3-media2.fl.yelpcdn.com/bphoto/-ptYT3...,False,https://www.yelp.com/biz/hidden-grounds-coffee...,16,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",4.5,"{'latitude': 40.72221, 'longitude': -74.04624}","[pickup, delivery]",NaN,"{'address1': '276 1st St', 'address2': None, '...",+12016100090,(201) 610-0090,4754.477120
998,0LtLFRGu0SYc3i4nGIabmA,the-bagel-shoppe-hackensack,The Bagel Shoppe,https://s3-media2.fl.yelpcdn.com/bphoto/6C07wa...,False,https://www.yelp.com/biz/the-bagel-shoppe-hack...,103,"[{'alias': 'bagels', 'title': 'Bagels'}, {'ali...",3.5,"{'latitude': 40.89908, 'longitude': -74.05611}","[pickup, delivery]",$,"{'address1': '447 Passaic St', 'address2': '',...",+12013422626,(201) 342-2626,22150.993340
999,QAGFu7pqCAZUJkYkUB5pjg,urbanspace-vanderbilt-new-york-3,Urbanspace Vanderbilt,https://s3-media3.fl.yelpcdn.com/bphoto/eMFzy4...,False,https://www.yelp.com/biz/urbanspace-vanderbilt...,692,"[{'alias': 'food_court', 'title': 'Food Court'}]",4.0,"{'latitude': 40.75445371780593, 'longitude': -...","[pickup, delivery]",$$,"{'address1': '230 Park Ave', 'address2': '', '...",+16467470810,(646) 747-0810,5649.399350


In [26]:
# check for duplicate ID's 
final_df.duplicated(subset='id').sum()

12

In [27]:
## Drop duplicate ids and confirm there are no more duplicates
final_df = final_df.drop_duplicates(subset='id')
final_df.duplicated(subset='id').sum()

0

In [28]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_NY_bagels.csv.gz', compression='gzip',index=False)